# Imports

In [2]:
!pip install aeon lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
from aeon.datasets import load_classification

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset, Dataset
from torchvision import datasets, transforms
from torchvision.models.resnet import Bottleneck, BasicBlock, conv1x1

import lightning as L

import sklearn
from sklearn.model_selection import train_test_split

In [17]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import sys
sys.path.append('/content/drive/MyDrive/FYP')

Mounted at /content/drive/


In [18]:
!git -C drive/MyDrive/FYP status

Refresh index: 100% (3/3), done.
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	ContrastiveClustering/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [24]:
!git -C drive/MyDrive/FYP pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 1), reused 4 (delta 1), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 427 bytes | 7.00 KiB/s, done.
From https://github.com/Sampie314/fyp
   0a46593..1e35a79  main       -> origin/main
Updating 0a46593..1e35a79
Fast-forward
 ContrastiveClustering/ContrastiveCluster.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [19]:
from ContrastiveClustering import ContrastiveCluster as CC

In [20]:
SEED = 69
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

# Load Benchmark Data

## Share Price Increase

The problem here is to predict whether a share price will show an exceptional rise after quarterly announcement of the Earning Per Share based on the price movement of that share price on the proceeding 60 days? The data was formatted by Vlad Pazenuks as part of his third year project. Daily price data on NASDAQ 100 companies was extracted from a Kaggle data set. Reporting dates of these companies were obtained from NASDAQ.com. Each data is the percentage change of the close price from the day before. Each case is a series of 60 day data. The target class is is defined as 0 = price did not increase after company report release by more than 5 percent 1 = price increased after company report release by more than 5 percent There are 1931 cases, 1326 class 0 and 605 class 1.

In [ ]:
share_price_increase = load_classification('SharePriceIncrease')

In [ ]:
print(len(share_price_increase))
share_data, share_labels = share_price_increase
# share_labels = share_labels.astype('int')
# share_data = torch.tensor(share_data)
# share_labels = torch.tensor(share_labels)

2


In [ ]:
share_labels

array(['0', '0', '0', ..., '1', '1', '1'], dtype='<U1')

In [ ]:
dataset = TensorDataset(share_data, share_labels)
dataloader = DataLoader(dataset)

In [ ]:
len(share_data)

1931

In [ ]:
len(share_data)

1931

In [ ]:
dataset

In [ ]:
len(share_data[0][0])

60

In [ ]:
dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])

In [ ]:
share_data

array([[[-4.14537643e+00, -7.70140984e-01,  2.00994691e+00, ...,
          5.79414027e-01,  6.43851347e-01,  8.75415202e-01]],

       [[-7.49062935e-01,  9.43390794e-01, -1.86911257e-01, ...,
          0.00000000e+00, -5.01675748e-01,  2.68908117e+00]],

       [[ 4.92917295e+00,  1.06292335e+00,  7.73693641e-01, ...,
         -4.89453986e-01, -6.91255106e-01, -1.28505329e+00]],

       ...,

       [[-7.21991939e-01,  7.44954318e-02, -1.46049916e+00, ...,
         -2.89349551e+00, -1.68077088e+00,  1.56116505e-01]],

       [[-5.66930304e-01, -8.26156696e-01, -2.81597423e-01, ...,
         -9.97105542e-01,  6.17291907e-01, -5.05867186e-01]],

       [[-7.74558560e-01,  5.83855185e-01, -4.22694555e-03, ...,
          2.99329532e-01,  3.00461488e-01, -4.03839405e-03]]])

In [ ]:
share_labels

array(['0', '0', '0', ..., '1', '1', '1'], dtype='<U1')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(share_data, share_labels, test_size = 0.6, random_state = SEED)

## Fashion MNIST

In [12]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
full_dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])

kwargs = {}
data_loader = DataLoader(full_dataset, batch_size=2, shuffle=True, **kwargs)

In [13]:
for (x_i, x_j) in data_loader:
    print(x_i)
    print(x_j)
    # print(_)
    break

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]]])
tensor([8, 0])


In [6]:
fmnist = datasets.FashionMNIST(root='./data', train=True, download=True)

In [11]:
transforms.ToTensor()(fmnist[0][0]).shape

torch.Size([1, 28, 28])

In [22]:
fmnist[0][0]

In [25]:
def get_mean_std(loader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in loader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

class ContrastiveFashionMNIST(Dataset):
    def __init__(self, root, train=True, transform=None, download=True):
        self.fashion_mnist = datasets.FashionMNIST(root=root, train=train, download=download)
        self.transform = transform

    def __getitem__(self, index):
        img, target = self.fashion_mnist[index]
        img1 = self.transform(img)
        img2 = self.transform(img)
        return (img1, img2), target

    def __len__(self):
        return len(self.fashion_mnist)

def load_fashion_mnist(batch_size, use_cuda=False):
    # First, create a DataLoader without normalization
    initial_transform = transforms.Compose([
        transforms.ToTensor()
    ])

    initial_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=initial_transform)
    initial_loader = DataLoader(initial_dataset, batch_size=batch_size, num_workers=2, shuffle=False)

    # Calculate mean and std
    mean, std = get_mean_std(initial_loader)
    # print(f"Calculated mean: {mean.item():.4f}, std: {std.item():.4f}")

    # Now create the full transform with the calculated mean and std
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((mean.item(),), (std.item(),))
    ])

    train_dataset = ContrastiveFashionMNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = ContrastiveFashionMNIST(root='./data', train=False, transform=transform, download=True)
    full_dataset = torch.utils.data.ConcatDataset([train_dataset, test_dataset])

    kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
    data_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True, drop_last=True, **kwargs)

    return data_loader


## Heterogeneity Human Activity Recognition (HHAR)

# Models

## Contrastive Clustering

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print('Using GPU')

# Hyperparameters
batch_size = 256
feature_dim = 128
class_num = 10
learning_rate = 0.0003
num_epochs = 100
instance_temperature = 0.5
cluster_temperature = 1.0

# Create backbone network
backbone = CC.ResNetFashionMNIST(block=BasicBlock, layers=[2, 2, 2, 2])  # ResNet18

model = CC.ContrastiveClusteringNetwork(backbone, feature_dim, class_num).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

criterion_instance = CC.InstanceLoss(batch_size, instance_temperature, device).to(device)
criterion_cluster = CC.ClusterLoss(class_num, cluster_temperature, device).to(device)

# Load Fashion-MNIST dataset
data_loader = load_fashion_mnist(batch_size, use_cuda=torch.cuda.is_available())

for epoch in range(num_epochs):
    loss = CC.train(model, data_loader, optimizer, criterion_instance, criterion_cluster, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}")


Epoch [1/100], Loss: 6.5200
Epoch [2/100], Loss: 6.3705
Epoch [3/100], Loss: 6.3565
Epoch [4/100], Loss: 6.3480


## Twin Contrastive Learning

## SOM

## Structural Deep Clustering Network